In [1]:
import pandas as pd
import pandas as np
import psycopg2
from psycopg2 import sql
import requests
import json
import pprint
import os
from ast import literal_eval

In [3]:
CKAN_URL = 'http://localhost/'
API_KEY = '64c8143c-6dee-4037-a8a4-3e3225c1b03a'

## CKAN API Functions:

In [4]:
def ckan_create_organization(org_dict, ckan_url, api_key, upload_image=False):
    '''
        Create new organization in ckan
    '''
    body_json = json.dumps(org_dict)  #reformat
    req = requests.post(ckan_url+'/api/action/organization_create', headers={'User-Agent':"Magic Browser", 'Authorization':api_key, "Content-Type": "application/json"}, data=body_json)
    assert req.status_code == 200             #check the request status (ckan always replies with 200)
    response_dict = json.loads(req.text)      #check the actual response 'success' status
    assert response_dict['success'] is True

    print('----- org: ' + org_dict['name'] + ' created')
    return response_dict['result']

In [ ]:
# def ckan_delete_organization(org_dict, ckan_url, api_key, upload_image=False):
#     '''
#         Create new organization in ckan
#         Input
#             org_dict : python dictionary which has {'key':'value'} ~ ckan org metadata
#             upload_image : specify whether to upload an image
#         *** Note that the org/group images need to be uploaded to /var/lib/ckan/default/storage/uploads/group

#     '''
#     body_json = json.dumps(org_dict)  #reformat
#     req = requests.post(ckan_url+'/api/action/organization_create', headers={'User-Agent':"Magic Browser", 'Authorization':api_key, "Content-Type": "application/json"}, data=body_json)
#     assert req.status_code == 200             #check the request status (ckan always replies with 200)
#     response_dict = json.loads(req.text)      #check the actual response 'success' status
#     assert response_dict['success'] is True

#     print('----- org: ' + org_dict['name'] + ' created')
#     return response_dict['result']

In [5]:
def ckan_create_user(user_dict, ckan_url, api_key, upload_image=False):
    '''
        Create new user in ckan
    '''
    body_json = json.dumps(user_dict)  #reformat
    req = requests.post(ckan_url+'/api/action/user_create', headers={'User-Agent':"Magic Browser", 'Authorization':api_key, "Content-Type": "application/json"}, data=body_json)
    print(req.text)
    assert req.status_code == 200             #check the request status (ckan always replies with 200)
    response_dict = json.loads(req.text)      #check the actual response 'success' status
    assert response_dict['success'] is True

    return response_dict['result']

In [6]:
def ckan_update_user(user_update_dict, ckan_url, api_key, upload_image=False):
    '''
        Update user in ckan
    '''
    print(user_update_dict)
    body_json = json.dumps(user_update_dict)  #reformat
    req = requests.post(ckan_url+'/api/action/user_update', headers={'User-Agent':"Magic Browser", 'Authorization':api_key, "Content-Type": "application/json"}, data=body_json)
    print(req.text)
    assert req.status_code == 200             #check the request status (ckan always replies with 200)
    response_dict = json.loads(req.text)      #check the actual response 'success' status
    assert response_dict['success'] is True

    return response_dict['result']

In [7]:
def ckan_delete_user(user_id_dict, ckan_url, api_key):
    '''
        Delete User in ckan
    '''
    body_json = json.dumps(user_id_dict)  #reformat
    req = requests.post(ckan_url+'/api/action/user_delete', headers={'User-Agent':"Magic Browser", 'Authorization':api_key, "Content-Type": "application/json"}, data=body_json)
    print(req.text)
    assert req.status_code == 200             #check the request status (ckan always replies with 200)
    response_dict = json.loads(req.text)      #check the actual response 'success' status
    assert response_dict['success'] is True

    return response_dict['result']

In [45]:
def ckan_assign_org_member(org_member_dict, ckan_url, api_key):
    '''
        Assign member to organization
    '''
    body_json = json.dumps(org_member_dict)  #reformat
    req = requests.post(ckan_url+'/api/action/organization_member_create', headers={'User-Agent':"Magic Browser", 'Authorization':api_key, "Content-Type": "application/json"}, data=body_json)
    print(req.text)
    assert req.status_code == 200             #check the request status (ckan always replies with 200)
    response_dict = json.loads(req.text)      #check the actual response 'success' status
    assert response_dict['success'] is True

    return response_dict['result']

## Create Organizations:

In [9]:
df_org = pd.read_excel("depa_organizations.xlsx")

In [10]:
df_org

,org_id,org_name,description
0,corporate-strategy-department,ฝ่ายกลยุทธ์องค์กร,NaN
1,office-administration-department,ฝ่ายอำนวยการสำนักงาน,NaN
2,policy-and-strategy-department,ฝ่ายนโยบายและยุทศาสตร์,NaN
3,smart-city-department,ฝ่ายส่งเสริมเมืองอัจฉริยะ,NaN
4,digital-industry-promotion-department,ฝ่ายส่งเสริมอุตสาหกรรมดิจิทัล,NaN
5,finanfe-and-central-administration-department,ฝ่ายบริหารการคลังและบริหารกลาง,NaN
6,iot-institute,สถาบันไอโอทีและนวัตกรรมดิจิทัล,เป็นองค์กรหลักในการขับเคลื่อนการพัฒนาเทคโนโลยี...
7,dmd,ฝ่ายสงเสริมแพลตฟอร์มและบริการดิจิทัล,NaN
8,organization-and-human-resources-management-de...,ฝ่ายบริหารทรัพยากรองค์กรและบุคคล,NaN
9,digital-platform-and-service-promotion-department,ฝ่ายสงเสริมแพลตฟอร์มและบริการดิจิทัล,NaN


In [11]:
df_org_master = df_org.copy()

In [12]:
# เปลี่ยนชื่อ col
df_org_master.rename(columns = {"org_id":"name", "org_name": "title"},inplace = True)

In [31]:
df_org_master

,name,title,description
0,corporate-strategy-department,ฝ่ายกลยุทธ์องค์กร,NaN
1,office-administration-department,ฝ่ายอำนวยการสำนักงาน,NaN
2,policy-and-strategy-department,ฝ่ายนโยบายและยุทศาสตร์,NaN
3,smart-city-department,ฝ่ายส่งเสริมเมืองอัจฉริยะ,NaN
4,digital-industry-promotion-department,ฝ่ายส่งเสริมอุตสาหกรรมดิจิทัล,NaN
5,finanfe-and-central-administration-department,ฝ่ายบริหารการคลังและบริหารกลาง,NaN
6,iot-institute,สถาบันไอโอทีและนวัตกรรมดิจิทัล,เป็นองค์กรหลักในการขับเคลื่อนการพัฒนาเทคโนโลยี...
7,dmd,ฝ่ายสงเสริมแพลตฟอร์มและบริการดิจิทัล,NaN
8,organization-and-human-resources-management-de...,ฝ่ายบริหารทรัพยากรองค์กรและบุคคล,NaN
9,digital-platform-and-service-promotion-department,ฝ่ายสงเสริมแพลตฟอร์มและบริการดิจิทัล,NaN


In [13]:
df_org_master.to_json('org_master.json', force_ascii=False, orient='records')

In [14]:
with open('org_master.json', encoding="utf8") as readfile:
    org_master_dict =  json.load(readfile)
org_master_dict

[{'name': 'corporate-strategy-department',
  'title': 'ฝ่ายกลยุทธ์องค์กร',
  'description': None},
 {'name': 'office-administration-department',
  'title': 'ฝ่ายอำนวยการสำนักงาน',
  'description': None},
 {'name': 'policy-and-strategy-department',
  'title': 'ฝ่ายนโยบายและยุทศาสตร์',
  'description': None},
 {'name': 'smart-city-department',
  'title': 'ฝ่ายส่งเสริมเมืองอัจฉริยะ',
  'description': None},
 {'name': 'digital-industry-promotion-department',
  'title': 'ฝ่ายส่งเสริมอุตสาหกรรมดิจิทัล',
  'description': None},
 {'name': 'finanfe-and-central-administration-department',
  'title': 'ฝ่ายบริหารการคลังและบริหารกลาง',
  'description': None},
 {'name': 'iot-institute',
  'title': 'สถาบันไอโอทีและนวัตกรรมดิจิทัล',
  'description': 'เป็นองค์กรหลักในการขับเคลื่อนการพัฒนาเทคโนโลยี นวัตกรรม และอุตสาหกรรมดิจิทัล เพื่อมุ่งสู่การเป็น “ศูนย์กลางด้านการออกแบบและพัฒนาเทคโนโลยีและนวัตกรรมดิจิทัลขั้นสูงแห่งภูมิภาคอาเซียน (ASEAN Digital Hub)”'},
 {'name': 'dmd',
  'title': 'ฝ่ายสงเสริมแพลตฟอร์มแ

In [15]:
for org in org_master_dict:
    try:
        result = ckan_create_organization(org, CKAN_URL, API_KEY)
        print(result)
    except:
        pass

----- org: corporate-strategy-department created
{'display_name': 'ฝ่ายกลยุทธ์องค์กร', 'description': '', 'image_display_url': '', 'package_count': 0, 'created': '2023-09-14T15:27:53.419939', 'name': 'corporate-strategy-department', 'is_organization': True, 'state': 'active', 'extras': [], 'image_url': '', 'groups': [], 'type': 'organization', 'title': 'ฝ่ายกลยุทธ์องค์กร', 'num_followers': 0, 'id': '7bcd469f-b068-4516-bb8e-cc628c707670', 'tags': [], 'approval_status': 'approved'}
----- org: office-administration-department created
{'display_name': 'ฝ่ายอำนวยการสำนักงาน', 'description': '', 'image_display_url': '', 'package_count': 0, 'created': '2023-09-14T15:27:53.559440', 'name': 'office-administration-department', 'is_organization': True, 'state': 'active', 'extras': [], 'image_url': '', 'groups': [], 'type': 'organization', 'title': 'ฝ่ายอำนวยการสำนักงาน', 'num_followers': 0, 'id': '5b091aac-be93-49cb-a308-885741b6e8c3', 'tags': [], 'approval_status': 'approved'}
----- org: policy-

## Create user accounts:

In [16]:
df_user = pd.read_excel("user_org_role.xlsx")

C:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [17]:
df_user

,ชื่อ,สกุล,email,หน่วยงาน,role
0,Kanyawee,Pornsawangdee,kanyawee.po@depa.or.th,สถาบันส่งเสริมการวิเคราะห์และบริหารข้อมูลขนาดใ...,member
1,Kanyawee,Pornsawangdee,kanyawee.po@depa.or.th,สถาบันส่งเสริมการวิเคราะห์และบริหารข้อมูลขนาดใ...,editor
2,Name1,Surname1,name1.su@depa.pr.th,ฝ่ายกลยุทธ์องค์กร,member
3,Name2,Surname2,name2.su@depa.pr.th,ฝ่ายกลยุทธ์องค์กร,member
4,Name3,Surname3,name3.su@depa.pr.th,ฝ่ายกลยุทธ์องค์กร,member
5,Name4,Surname4,name4.su@depa.pr.th,ฝ่ายกลยุทธ์องค์กร,member
6,Name5,Surname5,name5.su@depa.pr.th,ฝ่ายกลยุทธ์องค์กร,member
7,Name6,Surname6,name6.su@depa.pr.th,ฝ่ายกลยุทธ์องค์กร,member


In [18]:
df_user_org = pd.merge(df_user, df_org[["org_id","org_name"]], left_on="หน่วยงาน", right_on="org_name", how='left')
df_user_org

,ชื่อ,สกุล,email,หน่วยงาน,role,org_id,org_name
0,Kanyawee,Pornsawangdee,kanyawee.po@depa.or.th,สถาบันส่งเสริมการวิเคราะห์และบริหารข้อมูลขนาดใ...,member,gbdi,สถาบันส่งเสริมการวิเคราะห์และบริหารข้อมูลขนาดใ...
1,Kanyawee,Pornsawangdee,kanyawee.po@depa.or.th,สถาบันส่งเสริมการวิเคราะห์และบริหารข้อมูลขนาดใ...,editor,gbdi,สถาบันส่งเสริมการวิเคราะห์และบริหารข้อมูลขนาดใ...
2,Name1,Surname1,name1.su@depa.pr.th,ฝ่ายกลยุทธ์องค์กร,member,corporate-strategy-department,ฝ่ายกลยุทธ์องค์กร
3,Name2,Surname2,name2.su@depa.pr.th,ฝ่ายกลยุทธ์องค์กร,member,corporate-strategy-department,ฝ่ายกลยุทธ์องค์กร
4,Name3,Surname3,name3.su@depa.pr.th,ฝ่ายกลยุทธ์องค์กร,member,corporate-strategy-department,ฝ่ายกลยุทธ์องค์กร
5,Name4,Surname4,name4.su@depa.pr.th,ฝ่ายกลยุทธ์องค์กร,member,corporate-strategy-department,ฝ่ายกลยุทธ์องค์กร
6,Name5,Surname5,name5.su@depa.pr.th,ฝ่ายกลยุทธ์องค์กร,member,corporate-strategy-department,ฝ่ายกลยุทธ์องค์กร
7,Name6,Surname6,name6.su@depa.pr.th,ฝ่ายกลยุทธ์องค์กร,member,corporate-strategy-department,ฝ่ายกลยุทธ์องค์กร


In [19]:
df_user_org["id"] = df_user_org["email"].str.split("@").str[0].str.replace('.', '_')
df_user_org["name"] = df_user_org["email"].str.split("@").str[0].str.replace('.', '_')

In [20]:
df_user_org

,ชื่อ,สกุล,email,หน่วยงาน,role,org_id,org_name,id,name
0,Kanyawee,Pornsawangdee,kanyawee.po@depa.or.th,สถาบันส่งเสริมการวิเคราะห์และบริหารข้อมูลขนาดใ...,member,gbdi,สถาบันส่งเสริมการวิเคราะห์และบริหารข้อมูลขนาดใ...,kanyawee_po,kanyawee_po
1,Kanyawee,Pornsawangdee,kanyawee.po@depa.or.th,สถาบันส่งเสริมการวิเคราะห์และบริหารข้อมูลขนาดใ...,editor,gbdi,สถาบันส่งเสริมการวิเคราะห์และบริหารข้อมูลขนาดใ...,kanyawee_po,kanyawee_po
2,Name1,Surname1,name1.su@depa.pr.th,ฝ่ายกลยุทธ์องค์กร,member,corporate-strategy-department,ฝ่ายกลยุทธ์องค์กร,name1_su,name1_su
3,Name2,Surname2,name2.su@depa.pr.th,ฝ่ายกลยุทธ์องค์กร,member,corporate-strategy-department,ฝ่ายกลยุทธ์องค์กร,name2_su,name2_su
4,Name3,Surname3,name3.su@depa.pr.th,ฝ่ายกลยุทธ์องค์กร,member,corporate-strategy-department,ฝ่ายกลยุทธ์องค์กร,name3_su,name3_su
5,Name4,Surname4,name4.su@depa.pr.th,ฝ่ายกลยุทธ์องค์กร,member,corporate-strategy-department,ฝ่ายกลยุทธ์องค์กร,name4_su,name4_su
6,Name5,Surname5,name5.su@depa.pr.th,ฝ่ายกลยุทธ์องค์กร,member,corporate-strategy-department,ฝ่ายกลยุทธ์องค์กร,name5_su,name5_su
7,Name6,Surname6,name6.su@depa.pr.th,ฝ่ายกลยุทธ์องค์กร,member,corporate-strategy-department,ฝ่ายกลยุทธ์องค์กร,name6_su,name6_su


In [21]:
df_user_org["fullname"] = df_user_org["ชื่อ"] + ' ' + df_user_org["สกุล"]
df_user_org["password"] = "ckan1234"
df_user_org["about"] = ""
df_user_org["state"] = "active"

In [22]:
df_user_org

,ชื่อ,สกุล,email,หน่วยงาน,role,org_id,org_name,id,name,fullname,password,about,state
0,Kanyawee,Pornsawangdee,kanyawee.po@depa.or.th,สถาบันส่งเสริมการวิเคราะห์และบริหารข้อมูลขนาดใ...,member,gbdi,สถาบันส่งเสริมการวิเคราะห์และบริหารข้อมูลขนาดใ...,kanyawee_po,kanyawee_po,Kanyawee Pornsawangdee,ckan1234,,active
1,Kanyawee,Pornsawangdee,kanyawee.po@depa.or.th,สถาบันส่งเสริมการวิเคราะห์และบริหารข้อมูลขนาดใ...,editor,gbdi,สถาบันส่งเสริมการวิเคราะห์และบริหารข้อมูลขนาดใ...,kanyawee_po,kanyawee_po,Kanyawee Pornsawangdee,ckan1234,,active
2,Name1,Surname1,name1.su@depa.pr.th,ฝ่ายกลยุทธ์องค์กร,member,corporate-strategy-department,ฝ่ายกลยุทธ์องค์กร,name1_su,name1_su,Name1 Surname1,ckan1234,,active
3,Name2,Surname2,name2.su@depa.pr.th,ฝ่ายกลยุทธ์องค์กร,member,corporate-strategy-department,ฝ่ายกลยุทธ์องค์กร,name2_su,name2_su,Name2 Surname2,ckan1234,,active
4,Name3,Surname3,name3.su@depa.pr.th,ฝ่ายกลยุทธ์องค์กร,member,corporate-strategy-department,ฝ่ายกลยุทธ์องค์กร,name3_su,name3_su,Name3 Surname3,ckan1234,,active
5,Name4,Surname4,name4.su@depa.pr.th,ฝ่ายกลยุทธ์องค์กร,member,corporate-strategy-department,ฝ่ายกลยุทธ์องค์กร,name4_su,name4_su,Name4 Surname4,ckan1234,,active
6,Name5,Surname5,name5.su@depa.pr.th,ฝ่ายกลยุทธ์องค์กร,member,corporate-strategy-department,ฝ่ายกลยุทธ์องค์กร,name5_su,name5_su,Name5 Surname5,ckan1234,,active
7,Name6,Surname6,name6.su@depa.pr.th,ฝ่ายกลยุทธ์องค์กร,member,corporate-strategy-department,ฝ่ายกลยุทธ์องค์กร,name6_su,name6_su,Name6 Surname6,ckan1234,,active


In [23]:
df_user_org_master = df_user_org[["id","name","email",'password','fullname','about','state']]
df_user_org_master

,id,name,email,password,fullname,about,state
0,kanyawee_po,kanyawee_po,kanyawee.po@depa.or.th,ckan1234,Kanyawee Pornsawangdee,,active
1,kanyawee_po,kanyawee_po,kanyawee.po@depa.or.th,ckan1234,Kanyawee Pornsawangdee,,active
2,name1_su,name1_su,name1.su@depa.pr.th,ckan1234,Name1 Surname1,,active
3,name2_su,name2_su,name2.su@depa.pr.th,ckan1234,Name2 Surname2,,active
4,name3_su,name3_su,name3.su@depa.pr.th,ckan1234,Name3 Surname3,,active
5,name4_su,name4_su,name4.su@depa.pr.th,ckan1234,Name4 Surname4,,active
6,name5_su,name5_su,name5.su@depa.pr.th,ckan1234,Name5 Surname5,,active
7,name6_su,name6_su,name6.su@depa.pr.th,ckan1234,Name6 Surname6,,active


In [24]:
df_user_org_master.to_json('df_user_org_master.json', force_ascii=False, orient='records')

In [25]:
with open('df_user_org_master.json', encoding="utf8") as readfile:
    user_org_master_dict =  json.load(readfile)

In [26]:
user_org_master_dict

[{'id': 'kanyawee_po',
  'name': 'kanyawee_po',
  'email': 'kanyawee.po@depa.or.th',
  'password': 'ckan1234',
  'fullname': 'Kanyawee Pornsawangdee',
  'about': '',
  'state': 'active'},
 {'id': 'kanyawee_po',
  'name': 'kanyawee_po',
  'email': 'kanyawee.po@depa.or.th',
  'password': 'ckan1234',
  'fullname': 'Kanyawee Pornsawangdee',
  'about': '',
  'state': 'active'},
 {'id': 'name1_su',
  'name': 'name1_su',
  'email': 'name1.su@depa.pr.th',
  'password': 'ckan1234',
  'fullname': 'Name1 Surname1',
  'about': '',
  'state': 'active'},
 {'id': 'name2_su',
  'name': 'name2_su',
  'email': 'name2.su@depa.pr.th',
  'password': 'ckan1234',
  'fullname': 'Name2 Surname2',
  'about': '',
  'state': 'active'},
 {'id': 'name3_su',
  'name': 'name3_su',
  'email': 'name3.su@depa.pr.th',
  'password': 'ckan1234',
  'fullname': 'Name3 Surname3',
  'about': '',
  'state': 'active'},
 {'id': 'name4_su',
  'name': 'name4_su',
  'email': 'name4.su@depa.pr.th',
  'password': 'ckan1234',
  'fullna

In [27]:
for user in user_org_master_dict:
    try:
        _ = ckan_create_user(user, CKAN_URL, API_KEY)
    except:
        _ = ckan_update_user(user, CKAN_URL, API_KEY)

{"help": "http://localhost/api/3/action/help_show?name=user_create", "success": true, "result": {"email_hash": "8c6a87f90707b6f9563baf3af9fbc800", "about": "", "apikey": null, "display_name": "Kanyawee Pornsawangdee", "name": "kanyawee_po", "created": "2023-09-14T15:31:19.451219", "image_display_url": null, "id": "kanyawee_po", "sysadmin": false, "activity_streams_email_notifications": false, "state": "active", "image_url": null, "fullname": "Kanyawee Pornsawangdee", "email": "kanyawee.po@depa.or.th", "number_created_packages": 0}}
{"help": "http://localhost/api/3/action/help_show?name=user_create", "success": false, "error": {"name": ["\u0e0a\u0e37\u0e48\u0e2d\u0e25\u0e4a\u0e2d\u0e01\u0e2d\u0e34\u0e19\u0e19\u0e35\u0e49\u0e44\u0e21\u0e48\u0e27\u0e48\u0e32\u0e07"], "__type": "Validation Error"}}
{'id': 'kanyawee_po', 'name': 'kanyawee_po', 'email': 'kanyawee.po@depa.or.th', 'password': 'ckan1234', 'fullname': 'Kanyawee Pornsawangdee', 'about': '', 'state': 'active'}
{"help": "http://loc

## Delete user accounts:

In [ ]:
# df_user_org_master[['id']].to_json('df_user_id_list.json', force_ascii=False, orient='records')

In [ ]:
# with open('df_user_id_list.json', encoding="utf8") as readfile:
#     user_id_list_dict =  json.load(readfile)

In [ ]:
# for user_id in user_id_list_dict:
#     try:
#         _ = ckan_delete_user(user_id, CKAN_URL, API_KEY)
#     except:
#         pass

## Assign role and organization:

In [41]:
df_user_org_role = df_user_org[["name",'org_id',"role"]]

In [42]:
df_user_org_role

,name,org_id,role
0,kanyawee_po,gbdi,member
1,kanyawee_po,gbdi,editor
2,name1_su,corporate-strategy-department,member
3,name2_su,corporate-strategy-department,member
4,name3_su,corporate-strategy-department,member
5,name4_su,corporate-strategy-department,member
6,name5_su,corporate-strategy-department,member
7,name6_su,corporate-strategy-department,member


In [43]:
df_user_org_role.rename(columns = {"name":"username", "org_id": "id"},inplace = True)

C:\Users\User\AppData\Local\Temp\ipykernel_17028\166026037.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_user_org_role.rename(columns = {"name":"username", "org_id": "id"},inplace = True)


In [44]:
df_user_org_role

,username,id,role
0,kanyawee_po,gbdi,member
1,kanyawee_po,gbdi,editor
2,name1_su,corporate-strategy-department,member
3,name2_su,corporate-strategy-department,member
4,name3_su,corporate-strategy-department,member
5,name4_su,corporate-strategy-department,member
6,name5_su,corporate-strategy-department,member
7,name6_su,corporate-strategy-department,member


In [46]:
df_user_org_role.to_json('df_user_org_role.json', force_ascii=False, orient='records')

In [51]:
with open('df_user_org_role.json', encoding="utf8") as readfile:
    df_user_org_role =  json.load(readfile)

In [52]:
for user in df_user_org_role:
    try:
        _ = ckan_assign_org_member(user, CKAN_URL, API_KEY)
    except:
        _ = ckan_assign_org_member(user, CKAN_URL, API_KEY)

{"help": "http://localhost/api/3/action/help_show?name=organization_member_create", "success": true, "result": {"capacity": "member", "state": "active", "table_id": "kanyawee_po", "table_name": "user", "group_id": "424c82f8-d2d1-48d0-b894-4c33f3eb2002", "id": "7d48356a-5287-4a8c-9de2-e9a74d0721c6"}}
{"help": "http://localhost/api/3/action/help_show?name=organization_member_create", "success": true, "result": {"capacity": "editor", "state": "active", "table_id": "kanyawee_po", "table_name": "user", "group_id": "424c82f8-d2d1-48d0-b894-4c33f3eb2002", "id": "7d48356a-5287-4a8c-9de2-e9a74d0721c6"}}
{"help": "http://localhost/api/3/action/help_show?name=organization_member_create", "success": true, "result": {"capacity": "member", "state": "active", "table_id": "name1_su", "table_name": "user", "group_id": "7bcd469f-b068-4516-bb8e-cc628c707670", "id": "ac142a9c-2e93-444b-adc5-8d9cde617a16"}}
{"help": "http://localhost/api/3/action/help_show?name=organization_member_create", "success": true,